In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import stats

FILE_PATH = '/mnt/data/Metrics Dataset - Sheet15 - Metrics Dataset - Sheet15.csv'
CONF = 0.95
TARGET = None

df = pd.read_csv(FILE_PATH)
print(df.shape)
print(df.dtypes)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(num_cols)
if TARGET is None:
    TARGET = num_cols[0]
x = pd.to_numeric(df[TARGET], errors='coerce').dropna().values.astype(float)
N = x.size
print(TARGET, N)

In [ ]:
N = x.size
x_mean = x.mean()
x_var = x.var(ddof=1)
x_std = np.sqrt(x_var)
print(N, x_mean, x_var, x_std)

In [ ]:
bins = int(1 + 3.322 * np.log10(N))
hist, edges = np.histogram(x, bins=bins)
from scipy.stats import norm
cdf = norm.cdf
p = cdf((edges[1:]-x_mean)/x_std) - cdf((edges[:-1]-x_mean)/x_std)
exp = N * p
mask = exp > 0
chi2 = (((hist[mask]-exp[mask])**2)/exp[mask]).sum()
df_chi = mask.sum() - 3
p_chi = 1 - stats.chi2.cdf(chi2, df_chi) if df_chi>0 else np.nan

dago_stat, dago_p = stats.normaltest(x) if N>=8 else (np.nan, np.nan)
ks_stat, ks_p = stats.kstest((x - x_mean)/x_std, 'norm')

print(chi2, df_chi, p_chi)
print(dago_stat, dago_p)
print(ks_stat, ks_p)

In [ ]:
alpha = 1 - CONF
t_q = stats.t.ppf(1 - alpha/2, df=N-1)
mean_ci = (x_mean - t_q * x_std/np.sqrt(N), x_mean + t_q * x_std/np.sqrt(N))

chi_low = stats.chi2.ppf(alpha/2, df=N-1)
chi_high = stats.chi2.ppf(1 - alpha/2, df=N-1)
std_ci = (np.sqrt((N-1)*x_var/chi_high), np.sqrt((N-1)*x_var/chi_low))

print(mean_ci)
print(std_ci)

In [ ]:
G = np.max(np.abs(x - x_mean)) / x_std
t_crit = stats.t.ppf(1 - alpha/(2*N), df=N-2)
G_crit = ((N-1)/np.sqrt(N)) * np.sqrt(t_crit**2 / (N-2 + t_crit**2))
is_outlier = G > G_crit

print(G, G_crit, bool(is_outlier))
if is_outlier:
    idx = np.argmax(np.abs(x - x_mean))
    x2 = np.delete(x, idx)
    N2 = x2.size
    x2_mean = x2.mean()
    x2_var = x2.var(ddof=1)
    x2_std = np.sqrt(x2_var)
    t_q2 = stats.t.ppf(1 - alpha/2, df=N2-1)
    mean_ci2 = (x2_mean - t_q2 * x2_std/np.sqrt(N2), x2_mean + t_q2 * x2_std/np.sqrt(N2))
    chi_low2 = stats.chi2.ppf(alpha/2, df=N2-1)
    chi_high2 = stats.chi2.ppf(1 - alpha/2, df=N2-1)
    std_ci2 = (np.sqrt((N2-1)*x2_var/chi_high2), np.sqrt((N2-1)*x2_var/chi_low2))
    print(idx, x[idx], x2_mean, x2_std, mean_ci2, std_ci2)

In [ ]:
plt.figure()
plt.hist(x, bins='auto', density=True)
grid = np.linspace(x.min(), x.max(), 400)
from scipy.stats import norm
pdf = norm.pdf(grid, loc=x_mean, scale=x_std)
plt.plot(grid, pdf)
plt.tight_layout()
plt.show()

plt.figure()
stats.probplot(x, dist="norm", sparams=(x_mean, x_std), plot=plt)
plt.tight_layout()
plt.show()